In [1]:
import keras

Using TensorFlow backend.


目前，Keras 有三个后端实现可用: TensorFlow 后端，Theano 后端，CNTK 后端。

TensorFlow 是由 Google 开发的一个开源符号级张量操作框架。
Theano 是由蒙特利尔大学的 LISA Lab 开发的一个开源符号级张量操作框架。
CNTK 是由微软开发的一个深度学习开源工具包。
Cell Type->Markdown是能在這個筆記本中做註解

In [ ]:
%env KERAS_NACKEND=tensorflow

%env 可以用來設定環境變數

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from keras.datasets import mnist

讀入MNIST資料庫
此資料庫有一堆0-9的手寫數字圖庫。有6w筆訓練資料，1萬筆測試資料。他是NIST數據庫的簡化版，MNIST可以說是Deep Learning最有名的範例，他被Deep learning大師 Hinton稱為(機器學習的果蠅)
x為輸入，y為答案


In [ ]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()


In [ ]:
len(x_train)

In [ ]:
len(y_train)


In [ ]:
x_train[9487].shape

每筆輸入x就是一個手寫的0-9中一個數字的圖檔，大小為28*28。而輸出y當然就是正確答案。來看編號9487訓練資料的大小

In [ ]:
x_train[9487]


In [ ]:
plt.imshow(x_train[9487],cmap='Greys')

In [ ]:
y_train[9487]

2.3輸入格式整理
我們現在要用標準神經網路學手寫辨識，原來的每筆數據為28*28矩陣，但標準神經網路只能輸入平向量，也就是每次要28*28=784長的向量。因此我們要用reshape調整一下

In [ ]:
x_train = x_train.reshape(60000,784)

In [ ]:
x_train.shape

In [ ]:
x_test = x_test.reshape(10000,784)

In [ ]:
x_test.shape

2.4輸出格式整理
我們可能會想，我們想學的函數是f: R^784->R
但神經網路總有誤差 所以可能會得到f(x)=0.5
那這意思是可能是0，也有可能是1
於是我們會做"1-hot encoding",也就是
1 ->[0,1,0,0,0,0,0,0,0,0]
Keras其實已幫我們準備好套件了

In [ ]:
from keras.utils import np_utils

In [ ]:
y_train = np_utils.to_categorical(y_train,10)

In [ ]:
y_test= np_utils.to_categorical(y_test,10)

In [ ]:
y_train[9487]

3.打造地一個神經網路
我們決定了我們的函數是f:R^784->R^10
我們還要決定要幾個隱藏層，每層要幾個神經元，用哪個機方函數就可以了
3.1決定神經網路架構、讀入相關套件
假設我們要這麼做
使用2個hidden layers
每個hidden layer用500個神經元
activation function指名用sigmoid
於是從keras把相關套件讀入

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

dense就是你常用的的全连接层

3.2建構我們的神經網路
和以前做迴歸或機器學習一樣，我們就打開一個函數學習機，標準一層一層傳遞的神經網路叫sequential,於是我們打開一個空的神經網路

In [ ]:
model = Sequential()

我們每次用add去加一層，從一個隱藏層開始。而第一個隱藏層因為Keras當然猜不到輸入有幾個features,所以我們要告訴他

In [ ]:
model.add(Dense(500,input_dim=784))


In [ ]:
model.add(Activation('sigmoid'))


第二層hidden layer 因為前面輸出是500，所以只要一個參數引入

In [ ]:
model.add(Dense(500))
model.add(Activation('sigmoid'))

輸出有10個數字，所以齣齣的神經元是10
我們希望輸出加起來等於1，那這裡用softmax當activation 函數就ok

In [ ]:
model.add(Dense(10))
model.add(Activation('softmax'))

至此我們神經網路做完了

3.3組裝

和之前比較不一樣的是我們還要做compile才能正式把我們的神經網路建好，你可以發現我們還需要做幾件事:

決定使用的loss function,一般是mse

決定optimizer,我們用標準的SGD

設learning rate

為了一邊訓練一邊看結果，我們加設

metrics=['accuracy']

In [ ]:
model.compile(loss='mse',optimizer=SGD(lr=0.1),metrics=['accuracy'])

檢視我們的神經網路

In [ ]:
model.summary()

In [ ]:
#392500
784*500+500


In [ ]:
#250500 b=500
500*500+500

In [ ]:
#500
500*10+10

5.訓練你的第一個神經網路

現在訓練時，不能像以前一樣沒頭沒腦的把訓練資料送進去，這裡我們還有兩件事要做

一次要訓練幾筆資料(batch_size),我們就100筆調一次參數

這6萬筆資料一共要訓練幾次(epochs)，我們訓練個20次試試

In [ ]:
model.fit(x_train,y_train,batch_size=100,epochs=20)

我們用一個比較炫的方式來看訓練成果

In [ ]:
from ipywidgets import interact_manual

predict_classes會讓我們keras選10個輸出機率最大的那類

In [ ]:
predict = model.predict_classes(x_test)

In [ ]:
def test(測試):
    plt.imshow(x_test[測試].reshape(28,28),cmap="Greys")
    print("神經網路判斷為:",predict[測試])

In [ ]:
interact_manual(test,測試=(0,9999));

把神經網路存起來

之前沒裝pyh5要在終端機下安裝:

conda install h5py

In [ ]:
model_json =model.to_json()
open('handwrite_model.json','w').write(model_json)
model.save_weights('handwriting model weights.h5')